Model https://huggingface.co/mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization

Original colab https://colab.research.google.com/drive/1Ekd5pUeCX7VOrMx94_czTkwNtLN32Uyu?usp=sharing#scrollTo=oOoSrwWarJAC

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Judging from the original colab this resolves to BertTokenizer 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization")

# Judging from the original colab this resolves to EncoderDecoderModel
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization")

In [5]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel
rouge = datasets.load_metric("rouge")

test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = test_data.select(range(16))

batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids#.to("cuda")
    attention_mask = inputs.attention_mask#.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

Reusing dataset cnn_dailymail (/Users/Francis/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/1 [00:00<?, ?ba/s]

Score(precision=0.08611298567200036, recall=0.15913826219978974, fmeasure=0.11014522685157205)
